In [2]:
import mysql.connector
from mysql.connector import errorcode
import pandas as pd
import numpy as np


Convertimos archivo peliculas_api a Dataframe

In [112]:
archivo = 'peliculas_api.csv'

df_detalles_peliculas = pd.read_csv(archivo, index_col = 0)

df_detalles_peliculas.head(8)


,Tipo,Nombre,Año de Estreno,Mes de Estreno,ID de la Película,Genero
0,Movie,Pál Adrienn,2011,6,tt0146592,Accion/Drama
1,Movie,Oda az igazság,2010,2,tt0154039,Accion/Drama
2,Movie,A zöld sárkány gyermekei,2011,4,tt0162942,Accion/Drama
3,Movie,The Final Journey,2010,9,tt0230212,Accion/Drama
4,Movie,In My Sleep,2010,4,tt0326965,Accion/Drama
5,Movie,This Wretched Life,2010,7,tt0331312,Accion/Comedia/Drama
6,Movie,Blind Sided,Desconocido,Desconocido,tt0337882,Accion/Comedia/Drama
7,Movie,Anderson's Cross,2010,5,tt0393049,Accion/Comedia/Drama


Reemplazamos valores 'Desconocido' por 'nan' en todo el DataFrame

In [118]:

df_detalles_peliculas = df_detalles_peliculas.replace('Desconocido', np.nan)


In [119]:
df_detalles_peliculas.head(8)

,Tipo,Nombre,Año de Estreno,Mes de Estreno,ID de la Película,Genero
0,Movie,Pál Adrienn,2011,6,tt0146592,Accion/Drama
1,Movie,Oda az igazság,2010,2,tt0154039,Accion/Drama
2,Movie,A zöld sárkány gyermekei,2011,4,tt0162942,Accion/Drama
3,Movie,The Final Journey,2010,9,tt0230212,Accion/Drama
4,Movie,In My Sleep,2010,4,tt0326965,Accion/Drama
5,Movie,This Wretched Life,2010,7,tt0331312,Accion/Comedia/Drama
6,Movie,Blind Sided,None,None,tt0337882,Accion/Comedia/Drama
7,Movie,Anderson's Cross,2010,5,tt0393049,Accion/Comedia/Drama


 Convertir np.nan a None ya que SQL sólo acepta None

In [120]:
df_detalles_peliculas = df_detalles_peliculas.map(lambda x: None if pd.isna(x) else x)


In [121]:
df_detalles_peliculas.head(8)

,Tipo,Nombre,Año de Estreno,Mes de Estreno,ID de la Película,Genero
0,Movie,Pál Adrienn,2011,6,tt0146592,Accion/Drama
1,Movie,Oda az igazság,2010,2,tt0154039,Accion/Drama
2,Movie,A zöld sárkány gyermekei,2011,4,tt0162942,Accion/Drama
3,Movie,The Final Journey,2010,9,tt0230212,Accion/Drama
4,Movie,In My Sleep,2010,4,tt0326965,Accion/Drama
5,Movie,This Wretched Life,2010,7,tt0331312,Accion/Comedia/Drama
6,Movie,Blind Sided,None,None,tt0337882,Accion/Comedia/Drama
7,Movie,Anderson's Cross,2010,5,tt0393049,Accion/Comedia/Drama


Convertir el DataFrame en una lista de tuplas

In [122]:

data_tuplas = [tuple(x) for x in df_detalles_peliculas.itertuples(index=False)]
data_tuplas


[('Movie', 'Pál Adrienn', '2011', '6', 'tt0146592', 'Accion/Drama'),
 ('Movie', 'Oda az igazság', '2010', '2', 'tt0154039', 'Accion/Drama'),
 ('Movie',
  'A zöld sárkány gyermekei',
  '2011',
  '4',
  'tt0162942',
  'Accion/Drama'),
 ('Movie', 'The Final Journey', '2010', '9', 'tt0230212', 'Accion/Drama'),
 ('Movie', 'In My Sleep', '2010', '4', 'tt0326965', 'Accion/Drama'),
 ('Movie',
  'This Wretched Life',
  '2010',
  '7',
  'tt0331312',
  'Accion/Comedia/Drama'),
 ('Movie', 'Blind Sided', None, None, 'tt0337882', 'Accion/Comedia/Drama'),
 ('Movie',
  "Anderson's Cross",
  '2010',
  '5',
  'tt0393049',
  'Accion/Comedia/Drama'),
 ('Movie', 'Den milde smerte', '2014', '5', 'tt0396123', 'Accion/Drama'),
 ('Movie', 'Capture the Flag', None, None, 'tt0402887', 'Accion/Drama'),
 ('Movie', 'Utomlennye solntsem 2', '2010', '4', 'tt0403645', 'Accion/Drama'),
 ('Movie', 'Tekken', '2010', '3', 'tt0411951', 'Accion/Drama'),
 ('Movie', 'Disconnect', '2010', '8', 'tt0419692', 'Accion/Drama'),
 ('

Creamos Base de Datos SQL

In [130]:
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1', database='proyecto_cinemextract')


mycursor = cnx.cursor()
try:
    mycursor.execute("CREATE TABLE informacion_peliculas (ID_titulo INT AUTO_INCREMENT PRIMARY KEY,Tipo VARCHAR(5) NULL,Nombre VARCHAR(200) NULL, Año VARCHAR(4) NULL, Mes_estreno VARCHAR(10) NULL,ID_imdb VARCHAR(10) NULL,Genero VARCHAR(20) NULL)")
    print(mycursor)
except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

CMySQLCursor: CREATE TABLE informacion_peliculas (ID_t..


Creamos tabla principal e insertamos datos de 'peliculas_api'

In [131]:
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1', database='proyecto_cinemextract')


mycursor = cnx.cursor()
query = "INSERT INTO Informacion_peliculas (Tipo, Nombre, Año, Mes_estreno, ID_imdb, Genero) VALUES (%s, %s, %s, %s, %s, %s)"
try:
    mycursor.executemany(query, data_tuplas)
    cnx.commit()
    print(mycursor.rowcount, "records inserted.")
except mysql.connector.Error as err:
    print("Error inserting data:", err)
finally:
    if cnx.is_connected():
        mycursor.close()
        cnx.close()
        print("Connection closed.")

34975 records inserted.
Connection closed.


In [162]:
archivo_2 = 'fase-2-datos-peliculas-imdb-rotten.csv'

df_detalles_peliculas = pd.read_csv(archivo_2, index_col = 0)

df_detalles_peliculas.head(4)

,Puntuacion_IMDB,Puntuacion_Rotten_Tomatoes,Director,Guionistas,Argumento,Duracion
Nombre,,,,,,
Pál Adrienn,"6,7",62%,Ágnes Kocsis,Ágnes KocsisAndrea Roberti,"Una enfermera obesa y solitaria, que trabaja e...",2h 16min
Oda az igazság,"4,6",NaN,Miklós Jancsó,Miklós Jancsó,Un noble humanista del siglo XV es nombrado Re...,1h 40min
A zöld sárkány gyermekei,"6,9",NaN,Bence Miklauzic,Balázs MaruszkiBence MiklauzicGábor Németh,Un agente inmobiliario establece vínculos con ...,1h 29min
The Final Journey,"8,9",NaN,R.J. Adams,Diane C. AdamsR.J. AdamsPaul Kirby,Fotos de archivo y películas muestran las líne...,2h


In [164]:
df_detalles_peliculas= df_detalles_peliculas.replace('Desconocido', np.nan)
df_detalles_peliculas.head(8)

,Puntuacion_IMDB,Puntuacion_Rotten_Tomatoes,Director,Guionistas,Argumento,Duracion
Nombre,,,,,,
Pál Adrienn,"6,7",62%,Ágnes Kocsis,Ágnes KocsisAndrea Roberti,"Una enfermera obesa y solitaria, que trabaja e...",2h 16min
Oda az igazság,"4,6",NaN,Miklós Jancsó,Miklós Jancsó,Un noble humanista del siglo XV es nombrado Re...,1h 40min
A zöld sárkány gyermekei,"6,9",NaN,Bence Miklauzic,Balázs MaruszkiBence MiklauzicGábor Németh,Un agente inmobiliario establece vínculos con ...,1h 29min
The Final Journey,"8,9",NaN,R.J. Adams,Diane C. AdamsR.J. AdamsPaul Kirby,Fotos de archivo y películas muestran las líne...,2h
In My Sleep,"5,5",13%,Allen Wolf,Allen Wolf,A sleepwalking sexaholic masseur wakes up cove...,1h 44min
This Wretched Life,"7,8",NaN,Frankie J. Mosca,Frankie J. MoscaSal DeLuciaAnde Cira,"Awakening from a near fatal overdose, Chris em...",1h 39min
Blind Sided,"7,2",NaN,Douglas A. Raine,Jimmy LiftonPaulette Victor LiftonVirginia Per...,NaN,NaN
Anderson's Cross,"5,8",NaN,Jerome Elston Scott,Jerome Elston Scott,"Nick Anderson, Kevin Daniels, and Tracey Green...",1h 38min


In [165]:
df_detalles_peliculas = df_detalles_peliculas.map(lambda x: None if pd.isna(x) else x)
df_detalles_peliculas.head(8)

,Puntuacion_IMDB,Puntuacion_Rotten_Tomatoes,Director,Guionistas,Argumento,Duracion
Nombre,,,,,,
Pál Adrienn,"6,7",62%,Ágnes Kocsis,Ágnes KocsisAndrea Roberti,"Una enfermera obesa y solitaria, que trabaja e...",2h 16min
Oda az igazság,"4,6",None,Miklós Jancsó,Miklós Jancsó,Un noble humanista del siglo XV es nombrado Re...,1h 40min
A zöld sárkány gyermekei,"6,9",None,Bence Miklauzic,Balázs MaruszkiBence MiklauzicGábor Németh,Un agente inmobiliario establece vínculos con ...,1h 29min
The Final Journey,"8,9",None,R.J. Adams,Diane C. AdamsR.J. AdamsPaul Kirby,Fotos de archivo y películas muestran las líne...,2h
In My Sleep,"5,5",13%,Allen Wolf,Allen Wolf,A sleepwalking sexaholic masseur wakes up cove...,1h 44min
This Wretched Life,"7,8",None,Frankie J. Mosca,Frankie J. MoscaSal DeLuciaAnde Cira,"Awakening from a near fatal overdose, Chris em...",1h 39min
Blind Sided,"7,2",None,Douglas A. Raine,Jimmy LiftonPaulette Victor LiftonVirginia Per...,None,None
Anderson's Cross,"5,8",None,Jerome Elston Scott,Jerome Elston Scott,"Nick Anderson, Kevin Daniels, and Tracey Green...",1h 38min


In [166]:
data_tuplas_2 = [tuple(x) for x in df_detalles_peliculas.itertuples(index='Nombre')]
data_tuplas_2

[('Pál Adrienn',
  '6,7',
  '62%',
  'Ágnes Kocsis',
  'Ágnes KocsisAndrea Roberti',
  'Una enfermera obesa y solitaria, que trabaja en la sala de terminales de un hospital, se acuerda de su amiga de la infancia Adrienn Pal y quiere localizarla.',
  '2h 16min'),
 ('Oda az igazság',
  '4,6',
  None,
  'Miklós Jancsó',
  'Miklós Jancsó',
  'Un noble humanista del siglo XV es nombrado Rey de Hungría e impulsa una era de prosperidad.',
  '1h 40min'),
 ('A zöld sárkány gyermekei',
  '6,9',
  None,
  'Bence Miklauzic',
  'Balázs MaruszkiBence MiklauzicGábor Németh',
  'Un agente inmobiliario establece vínculos con un guardia que vive en un almacén que le han encargado vender. El jefe del guardia quiere bloquear la venta, pero le hace promesas para hacer realidad las ambiciones futbolísticas del guardia.',
  '1h 29min'),
 ('The Final Journey',
  '8,9',
  None,
  'R.J. Adams',
  'Diane C. AdamsR.J. AdamsPaul Kirby',
  'Fotos de archivo y películas muestran las líneas de ferrocarril que conducí

In [189]:
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1', database='proyecto_cinemextract')


mycursor = cnx.cursor()
try:
    mycursor.execute("CREATE TABLE detalles_peliculas (ID_titulo INT AUTO_INCREMENT PRIMARY KEY,Nombre VARCHAR(250),Puntuacion_IMDB VARCHAR(6) NULL,Puntuacion_Rotten_Tomatoes VARCHAR(4) NULL, Director VARCHAR(250) NULL, Guionista VARCHAR(250) NULL, Argumento VARCHAR(900) NULL,Duracion VARCHAR(10) NULL)")
    print(mycursor)
except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

CMySQLCursor: CREATE TABLE detalles_peliculas (ID_titu..


In [190]:
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1', database='proyecto_cinemextract')


mycursor = cnx.cursor()
query = "INSERT INTO detalles_peliculas (Nombre, Puntuacion_IMDB, Puntuacion_Rotten_Tomatoes, Director, Guionista, Argumento, Duracion ) VALUES (%s, %s, %s, %s, %s, %s,%s)"
try:
    mycursor.executemany(query, data_tuplas_2)
    cnx.commit()
    print(mycursor.rowcount, "records inserted.")
except mysql.connector.Error as err:
    print("Error inserting data:", err)
finally:
    if cnx.is_connected():
        mycursor.close()
        cnx.close()
        print("Connection closed.")

34973 records inserted.
Connection closed.


In [191]:
archivo_3 = 'Fase_4_datos_oscars.csv'

df_oscar = pd.read_csv(archivo_3, index_col = 0)

df_oscar.head(4)

,Mejor Película,Mejor Director,Mejor Actor,Mejor Actriz
Fecha,,,,
27 de febrero de 2011,The King's Speech,T. Hooper(The King's Speech),C. Firth(The King's Speech),N. Portman(Black Swan)
26 de febrero de 2012,The Artist,M. Hazanavicius(The Artist),J. Dujardin(The Artist),M. Streep(The Iron Lady)
24 de febrero de 2013,Argo,A. Lee(Life of Pi),D. Day-Lewis(Lincoln),J. Lawrence(Silver Linings Playbook)
2 de marzo de 2014,12 Years a Slave,A. Cuarón(Gravity),M. McConaughey(Dallas Buyers Club),C. Blanchett(Blue Jasmine)


In [194]:
data_tuplas_3 = [tuple(x) for x in df_oscar.itertuples(index='Fecha')]
data_tuplas_3

[('27 de febrero de 2011',
  "The King's Speech",
  "T. Hooper(The King's Speech)",
  "C. Firth(The King's Speech)",
  'N. Portman(Black Swan)'),
 ('26 de febrero de 2012',
  'The Artist',
  'M. Hazanavicius(The Artist)',
  'J. Dujardin(The Artist)',
  'M. Streep(The Iron Lady)'),
 ('24 de febrero de 2013',
  'Argo',
  'A. Lee(Life of Pi)',
  'D. Day-Lewis(Lincoln)',
  'J. Lawrence(Silver Linings Playbook)'),
 ('2 de marzo de 2014',
  '12 Years a Slave',
  'A. Cuarón(Gravity)',
  'M. McConaughey(Dallas Buyers Club)',
  'C. Blanchett(Blue Jasmine)'),
 ('22 de febrero de 2015',
  'Birdman or (The Unexpected Virtue of Ignorance)',
  'A. G. Iñárritu(Birdman or (The Unexpected Virtue of Ignorance))',
  'E. Redmayne(The Theory Of Everything)',
  'J.Moore(Still Alice)'),
 ('28 de febrero de 2016',
  'Spotlight',
  'A. G. Iñárritu(The Revenant)',
  'L. DiCaprio(The Revenant)',
  'B. Larson(Room)'),
 ('26 de febrero de 2017',
  'Moonlight',
  'D. Chazelle(La La Land)',
  'C. Affleck(Manchester 

In [195]:
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1', database='proyecto_cinemextract')


mycursor = cnx.cursor()
try:
    mycursor.execute("CREATE TABLE premios_oscar (fecha VARCHAR(25), mejor_pelicula VARCHAR(250),mejor_director VARCHAR(250) ,mejor_actor  VARCHAR(250) , mejor_actriz VARCHAR(250))")
    print(mycursor)
except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

CMySQLCursor: CREATE TABLE premios_oscar (fecha VARCHA..


In [196]:
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1', database='proyecto_cinemextract')


mycursor = cnx.cursor()
query = "INSERT INTO premios_oscar (fecha,  mejor_pelicula, mejor_director, mejor_actor, mejor_actriz  ) VALUES (%s, %s, %s, %s, %s)"
try:
    mycursor.executemany(query, data_tuplas_3)
    cnx.commit()
    print(mycursor.rowcount, "records inserted.")
except mysql.connector.Error as err:
    print("Error inserting data:", err)
finally:
    if cnx.is_connected():
        mycursor.close()
        cnx.close()
        print("Connection closed.")

14 records inserted.
Connection closed.


In [4]:
datos_fase3 = pd.read_csv('Fase_3_datos_actores.csv')
datos_fase3

,Unnamed: 0,nombre_peli,nombre_actxr,conocido_por,year_nacimiento,profesiones_string,premios
0,0,La vida secreta de Walter Mitty,Ben Stiller,"['Tropic Thunder: ¡Una guerra muy perra!', 'Zo...",1965,"Producción, Reparto, Dirección",18 premios y 69 nominaciones en total
1,1,La vida secreta de Walter Mitty,Kristen Wiig,"['La boda de mi mejor amiga', 'The Skeleton Tw...",1973,"Reparto, Producción, Guion",15 premios y 84 nominaciones en total
2,2,La vida secreta de Walter Mitty,Jon Daly,"['De-mentes criminales', 'Zoolander No. 2']",1977,"Reparto, Guion, Producción",3 nominaciones
3,3,La vida secreta de Walter Mitty,Kathryn Hahn,"['Afternoon Delight', 'Transparent']",1973,"Reparto, Producción, Guion",10 premios y 33 nominaciones en total
4,4,La vida secreta de Walter Mitty,Terence Bernie Hines,"['La vida secreta de Walter Mitty', 'I Think Y...",1969,"Reparto, Dirección, Guion",8 premios y 1 nominación
...,...,...,...,...,...,...,...
16808,3089,Chasing Cane: At What Cost,NaN,['N/A'],NaN,NaN,NaN
16809,3090,Chasing Cane: At What Cost,NaN,['N/A'],NaN,NaN,NaN
16810,3091,Chasing Cane: At What Cost,NaN,['N/A'],NaN,NaN,NaN
16811,3092,Chasing Cane: At What Cost,NaN,['N/A'],NaN,NaN,NaN


In [6]:
df_modificado = datos_fase3.drop(['Unnamed: 0'], axis=1)
df_modificado

,nombre_peli,nombre_actxr,conocido_por,year_nacimiento,profesiones_string,premios
0,La vida secreta de Walter Mitty,Ben Stiller,"['Tropic Thunder: ¡Una guerra muy perra!', 'Zo...",1965,"Producción, Reparto, Dirección",18 premios y 69 nominaciones en total
1,La vida secreta de Walter Mitty,Kristen Wiig,"['La boda de mi mejor amiga', 'The Skeleton Tw...",1973,"Reparto, Producción, Guion",15 premios y 84 nominaciones en total
2,La vida secreta de Walter Mitty,Jon Daly,"['De-mentes criminales', 'Zoolander No. 2']",1977,"Reparto, Guion, Producción",3 nominaciones
3,La vida secreta de Walter Mitty,Kathryn Hahn,"['Afternoon Delight', 'Transparent']",1973,"Reparto, Producción, Guion",10 premios y 33 nominaciones en total
4,La vida secreta de Walter Mitty,Terence Bernie Hines,"['La vida secreta de Walter Mitty', 'I Think Y...",1969,"Reparto, Dirección, Guion",8 premios y 1 nominación
...,...,...,...,...,...,...
16808,Chasing Cane: At What Cost,NaN,['N/A'],NaN,NaN,NaN
16809,Chasing Cane: At What Cost,NaN,['N/A'],NaN,NaN,NaN
16810,Chasing Cane: At What Cost,NaN,['N/A'],NaN,NaN,NaN
16811,Chasing Cane: At What Cost,NaN,['N/A'],NaN,NaN,NaN


In [7]:
df_modificado= df_modificado.map(lambda x: None if pd.isna(x) else x)
df_modificado

,nombre_peli,nombre_actxr,conocido_por,year_nacimiento,profesiones_string,premios
0,La vida secreta de Walter Mitty,Ben Stiller,"['Tropic Thunder: ¡Una guerra muy perra!', 'Zo...",1965,"Producción, Reparto, Dirección",18 premios y 69 nominaciones en total
1,La vida secreta de Walter Mitty,Kristen Wiig,"['La boda de mi mejor amiga', 'The Skeleton Tw...",1973,"Reparto, Producción, Guion",15 premios y 84 nominaciones en total
2,La vida secreta de Walter Mitty,Jon Daly,"['De-mentes criminales', 'Zoolander No. 2']",1977,"Reparto, Guion, Producción",3 nominaciones
3,La vida secreta de Walter Mitty,Kathryn Hahn,"['Afternoon Delight', 'Transparent']",1973,"Reparto, Producción, Guion",10 premios y 33 nominaciones en total
4,La vida secreta de Walter Mitty,Terence Bernie Hines,"['La vida secreta de Walter Mitty', 'I Think Y...",1969,"Reparto, Dirección, Guion",8 premios y 1 nominación
...,...,...,...,...,...,...
16808,Chasing Cane: At What Cost,None,['N/A'],None,None,None
16809,Chasing Cane: At What Cost,None,['N/A'],None,None,None
16810,Chasing Cane: At What Cost,None,['N/A'],None,None,None
16811,Chasing Cane: At What Cost,None,['N/A'],None,None,None


In [10]:
data_tuplas_4 = [tuple(x) for x in df_modificado.itertuples(index=False)]
data_tuplas_4

[('La vida secreta de Walter Mitty',
  'Ben Stiller',
  "['Tropic Thunder: ¡Una guerra muy perra!', 'Zoolander (Un descerebrado de moda)']",
  '1965',
  'Producción, Reparto, Dirección',
  '18 premios y 69 nominaciones en total'),
 ('La vida secreta de Walter Mitty',
  'Kristen Wiig',
  "['La boda de mi mejor amiga', 'The Skeleton Twins']",
  '1973',
  'Reparto, Producción, Guion',
  '15 premios y 84 nominaciones en total'),
 ('La vida secreta de Walter Mitty',
  'Jon Daly',
  "['De-mentes criminales', 'Zoolander No. 2']",
  '1977',
  'Reparto, Guion, Producción',
  '3 nominaciones'),
 ('La vida secreta de Walter Mitty',
  'Kathryn Hahn',
  "['Afternoon Delight', 'Transparent']",
  '1973',
  'Reparto, Producción, Guion',
  '10 premios y 33 nominaciones en total'),
 ('La vida secreta de Walter Mitty',
  'Terence Bernie Hines',
  "['La vida secreta de Walter Mitty', 'I Think You Should Leave with Tim Robinson']",
  '1969',
  'Reparto, Dirección, Guion',
  '8 premios y 1 nominación'),
 ('

In [12]:
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1', database='proyecto_cinemextract')


mycursor = cnx.cursor()
try:
    mycursor.execute("CREATE TABLE informacion_actores (ID_actor INT AUTO_INCREMENT PRIMARY KEY, nombre_pelicula VARCHAR(250),nombre_actor VARCHAR(250) ,trabajos_destacados  VARCHAR(250) , año_nacimiento VARCHAR(100), profesion VARCHAR(250), premios VARCHAR(100) )")
    print(mycursor)
except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

CMySQLCursor: CREATE TABLE informacion_actores (ID_act..


In [13]:
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1', database='proyecto_cinemextract')


mycursor = cnx.cursor()
query = "INSERT INTO informacion_actores ( nombre_pelicula ,nombre_actor  ,trabajos_destacados   , año_nacimiento , profesion , premios ) VALUES (%s, %s, %s, %s, %s, %s)"
try:
    mycursor.executemany(query, data_tuplas_4)
    cnx.commit()
    print(mycursor.rowcount, "records inserted.")
except mysql.connector.Error as err:
    print("Error inserting data:", err)
finally:
    if cnx.is_connected():
        mycursor.close()
        cnx.close()
        print("Connection closed.")

16813 records inserted.
Connection closed.
